In [1]:
from datetime import datetime
from IPython.display import display
from collections import deque

import requests
import pandas as pd

In [2]:
API = 'http://localhost:5600/api'
last_updated = {}

In [3]:
r = requests.get(f'{API}/0/buckets')
buckets = r.json()

In [4]:
afk_buckets = [b for b in buckets.values() if b['type'] == 'afkstatus']
title_buckets = [b for b in buckets.values() if b['type'] == 'currentwindow']
display(afk_buckets)
display(title_buckets)

[{'id': 'aw-watcher-afk_pdsk',
  'created': '2020-07-24T16:26:53.815234+00:00',
  'name': None,
  'type': 'afkstatus',
  'client': 'aw-watcher-afk',
  'hostname': 'pdsk',
  'last_updated': '2020-08-26T09:07:34.824000+00:00'},
 {'id': 'aw-watcher-afk_pavel-dsk',
  'created': '2020-06-10T16:39:16.423874+00:00',
  'name': None,
  'type': 'afkstatus',
  'client': 'aw-watcher-afk',
  'hostname': 'pavel-dsk',
  'last_updated': '2020-07-15T07:42:24.178000+00:00'}]

[{'id': 'aw-watcher-window_pdsk',
  'created': '2020-07-24T16:26:53.677257+00:00',
  'name': None,
  'type': 'currentwindow',
  'client': 'aw-watcher-window',
  'hostname': 'pdsk',
  'last_updated': '2020-08-26T09:07:34.111000+00:00'},
 {'id': 'aw-watcher-window_pavel-dsk',
  'created': '2020-06-10T16:39:16.280454+00:00',
  'name': None,
  'type': 'currentwindow',
  'client': 'aw-watcher-window',
  'hostname': 'pavel-dsk',
  'last_updated': '2020-07-15T07:42:23.336000+00:00'}]

In [9]:
def get_data(bucket_id, last_updated=None):
    params = {}
    if last_updated:
        params['start'] = last_updated
    r = requests.get(f'{API}/0/buckets/{bucket_id}/events')
    data = deque()
    for event in r.json():
        data.append({
            'id': event['id'],
            'duration': event['duration'],
            'timestamp': pd.Timestamp(event['timestamp']),
            **event['data']
        })
    df = pd.DataFrame(data)
    return df

get_data('aw-watcher-afk_pdsk', '2020-07-23T16:26:53.815234+00:00')

,id,duration,timestamp,status
0,203897,2326.279,2020-08-26 08:30:44.078000+00:00,not-afk
1,203426,4014.610,2020-08-25 17:35:42.773000+00:00,not-afk
2,203420,205.334,2020-08-25 17:32:17.439000+00:00,afk
3,202920,4481.431,2020-08-25 16:17:36.008000+00:00,not-afk
4,202918,1487.191,2020-08-25 15:52:48.817000+00:00,afk
...,...,...,...,...
439,841,4281.695,2020-07-24 15:27:41.067000+00:00,not-afk
440,840,1996.156,2020-07-24 14:54:24.911000+00:00,afk
441,837,20.169,2020-07-24 14:54:04.742000+00:00,not-afk
442,836,281.157,2020-07-24 14:49:23.585000+00:00,afk
